In [30]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

In [31]:
#data loading
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
data = pd.read_csv(url)
data

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121
...,...,...
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390


In [32]:
# normalization
scaler = MinMaxScaler()
passengers = scaler.fit_transform(data[['Passengers']].values)

In [33]:
# convert the data into labeld data
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

In [34]:
seq_length = 5
X, y = create_sequences(passengers, seq_length)

In [35]:
# tensor
X = torch.tensor(X, dtype=torch.float32)  # shape: (samples, seq_len, 1)
y = torch.tensor(y, dtype=torch.float32)  # shape: (samples, 1)

In [36]:
# model
class RNNModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=10, output_size=1):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)   # RNN processes sequence
        out = out[:, -1, :]    # take output at last timestep
        out = self.fc(out)     # map hidden → prediction
        return out

In [37]:
# create a object for model
model=RNNModel()

In [38]:
# loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [39]:
epochs = 200
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [20/200], Loss: 0.0237
Epoch [40/200], Loss: 0.0076
Epoch [60/200], Loss: 0.0062
Epoch [80/200], Loss: 0.0056
Epoch [100/200], Loss: 0.0052
Epoch [120/200], Loss: 0.0049
Epoch [140/200], Loss: 0.0045
Epoch [160/200], Loss: 0.0042
Epoch [180/200], Loss: 0.0040
Epoch [200/200], Loss: 0.0038


In [40]:
# testing
with torch.no_grad():
    # last 5 months data
    test_input = torch.tensor(passengers[-5:], dtype=torch.float32).view(1, seq_length, 1)
    prediction = model(test_input).item()
    prediction = scaler.inverse_transform([[prediction]])  # scale back
    print("\nPredicted passengers for next month:", prediction[0][0])


Predicted passengers for next month: 437.81108725070953
